<a href="https://colab.research.google.com/github/lucia1970-student/Autism_Repo/blob/main/MFCC_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
cd /content/drive/MyDrive/Colab\ Notebooks/cours_python/FSCI2610/projet

/content/drive/MyDrive/Colab Notebooks/cours_python/FSCI2610/projet


In [28]:
!pip install --upgrade pip setuptools wheel

In [30]:
!pip install praat-parselmouth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 71.9 MB/s eta 0:00:00


In [37]:
import librosa
import numpy as np
import parselmouth
from scipy.stats import variation

# Load the audio file
def load_audio(file_path):
    y, sr = librosa.load(file_path, sr=None)
    return y, sr

# Compute MFCC features
def compute_mfcc(y, sr, n_mfcc=13):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfcc

# Compute F0 (fundamental frequency) using Parselmouth (Praat wrapper)
def compute_f0(file_path):
    sound = parselmouth.Sound(file_path)
    pitch = sound.to_pitch()
    f0 = pitch.selected_array['frequency']
    f0 = f0[f0 > 0]  # Remove zero or negative values
    return f0

# Compute Jitter (F0 variation)
def compute_jitter(f0):
    # Jitter is the variation in F0, measured as the relative standard deviation
    jitter = variation(f0) if len(f0) > 1 else 0
    return jitter

# Compute Shimmer (Amplitude variation)
def compute_shimmer(y):
    # Calculate short-term energy (amplitude variation)
    frame_length = 2048
    hop_length = 512
    energy = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)

    # Compute the variation of energy
    shimmer = variation(energy.flatten()) if len(energy.flatten()) > 1 else 0
    return shimmer

# Compute HNR (Harmonics-to-Noise Ratio)
def compute_hnr(file_path):
    sound = parselmouth.Sound(file_path)
    pitch = sound.to_pitch()
    harmonicity = sound.to_harmonicity()

    # HNR is the ratio of harmonic to noise components
    hnr = harmonicity.values.T
    return np.mean(hnr[hnr > 0])  # Only consider positive HNR values

# Main function to compute all features
def compute_voice_features(file_path):
    # Load the audio file
    y, sr = load_audio(file_path)

    # Compute MFCCs
    mfcc = compute_mfcc(y, sr)

    # Compute F0
    f0 = compute_f0(file_path)

    # Compute Jitter
    jitter = compute_jitter(f0)

    # Compute Shimmer
    shimmer = compute_shimmer(y)

    # Compute HNR
    hnr = compute_hnr(file_path)

    # Results
    features = {
        'MFCC': mfcc,
        'F0': f0,
        'Jitter': jitter,
        'Shimmer': shimmer,
        'HNR': hnr
    }

    return features

file_path = './wav/voice_samples/Normal_Voice.wav'
features = compute_voice_features(file_path)

# Print features
print("MFCCs:", features['MFCC'].shape)
print("Means F0 (Fundamental Frequency):", np.mean(features['F0']))
print("Jitter:", features['Jitter'])
print("Shimmer:", features['Shimmer'])
print("HNR:", features['HNR'])


MFCCs: (13, 2712)
Means F0 (Fundamental Frequency): 212.45936758708945
Jitter: 0.2010853914661789
Shimmer: 0.8738553
HNR: 14.17612320806539
